<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/5_Embedding_QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách xây dựng một pipeline QA nâng cao với EmbeddingRetriever**



# 1. Kiểm tra GPU runtime

In [1]:
%%bash

nvidia-smi

Sat Jul  1 13:22:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,faiss,inference]

# 3. Bật chế độ logging

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# 4. Khởi tạo DocumentStore FAISS

In [4]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


# 5. Chuẩn bị các file tài liệu

In [5]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http

doc_dir = "data/witcher"

fetch_archive_from_http(
    url="https://github.com/kynthesis/HaystackResearch/raw/main/witcher.zip",
    output_dir=doc_dir,
)

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO:haystack.utils.import_utils:Fetching from https://github.com/kynthesis/HaystackResearch/raw/main/witcher.zip to 'data/witcher'
INFO:haystack.utils.preprocessing:Converting data/witcher/3 - Blood of Elves.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/5 - Baptism of Fire.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/7 - The Lady of the Lake.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/6 - The Tower of the Swallow.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/2 - The Sword of Destiny.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/4 - Times of Contempt.txt
INFO:haystack.utils.preprocessing:Converting data/witcher/1 - The Last Wish.txt


# 6. Indexing các file tài liệu vào DocumentStore

In [6]:
from haystack.nodes import EmbeddingRetriever

document_store.write_documents(docs)

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)

document_store.update_embeddings(retriever)

Writing Documents:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.document_stores.faiss:Updating embeddings for 7 docs...


Updating Embedding:   0%|          | 0/7 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# 7. Tạo pipeline QA gồm Retriever và Reader

In [7]:
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


# 9. Đặt câu hỏi cho pipeline QA

In [22]:
prediction = pipe.run(
    query="Who is the instructor of Geralt?",
    params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples:   0%|          | 0/185 [00:00<?, ? Batches/s]

# 10. Nhận câu trả lời từ pipeline QA

In [23]:
from haystack.utils import print_answers

print_answers(prediction, details="medium")

'Query: Who is the instructor of Geralt?'
'Answers:'
[   {   'answer': 'Vesemir',
        'context': ' in fits of laughter watching the antics of the tied '
                   'bumble-\n'
                   'bee, until Vesemir, their tutor, caught them at it and '
                   'tanned their hides with a leather ',
        'score': 0.9451360702514648},
    {   'answer': 'de Vries',
        'context': 'the traitors to Garstang! Dethmold, offer you arm to the '
                   'great teacher de Vries. Go \n'
                   ' Steps. The smell of cinnamon and spikenard. \n'
                   " 'Your subordinates",
        'score': 0.9096723794937134},
    {   'answer': 'Tissaia de Vries',
        'context': ', gesturing pedantically \n'
                   'and ordering everything on hand. This is Tissaia de Vries, '
                   'great teacher, who educated tens of \n'
                   'sorceresses. Teaching them t',
        'score': 0.9018232822418213},
    {   'answe